# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,-3.27,93,0,2.57,US,1697219487
1,1,kingston,17.9970,-76.7936,33.47,65,20,7.72,JM,1697219239
2,2,tralee,52.2704,-9.7026,10.60,68,45,7.85,IE,1697219488
3,3,albany,42.6001,-73.9662,16.44,53,10,1.79,US,1697219488
4,4,chernyshevskiy,63.0128,112.4714,-2.25,83,100,1.72,RU,1697219489


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
#hvplot
humidity_plot = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          geo=True,
                                           size="Humidity",
                                           color= "City",
                                           tiles="OSM"
                                          )
                                            
# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] <= 30.5) & (city_data_df['Max Temp'] >= 23.8) 
                             & (city_data_df['Cloudiness'] <= 35) & (city_data_df['Wind Speed'] <= 10)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
    
# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,enewetak,11.3474,162.3373,28.15,74,29,2.71,MH,1697219494
17,17,kill devil hills,36.0307,-75.6760,24.00,66,0,4.63,US,1697219497
30,30,veraval,20.9000,70.3667,27.36,81,0,3.39,IN,1697219502
51,51,laguna,38.4210,-121.4238,25.19,48,0,1.54,US,1697219509
55,55,fort bragg,35.1390,-79.0060,26.47,45,0,3.09,US,1697219511


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,enewetak,MH,11.3474,162.3373,74,
17,kill devil hills,US,36.0307,-75.6760,66,
30,veraval,IN,20.9000,70.3667,81,
51,laguna,US,38.4210,-121.4238,48,
55,fort bragg,US,35.1390,-79.0060,45,
...,...,...,...,...,...,...
557,berbera,SO,10.4396,45.0143,67,
565,saint-francois,FR,46.4154,3.9054,43,
585,sirte,LY,31.2089,16.5887,64,
588,kapa'a,US,22.0752,-159.3190,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "limit": 1,
    "categories": "accommodation.hotel",
    "apiKey" : geoapify_key
        }
        

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng'] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
enewetak - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
veraval - nearest hotel: Shyam
laguna - nearest hotel: Holiday Inn Express & Suites
fort bragg - nearest hotel: Airborne Inn Lodging
al kharijah - nearest hotel: Kharga Hotel
dwarka - nearest hotel: The Dwarika Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kumo - nearest hotel: No hotel found
makuyuni - nearest hotel: No hotel found
chuhar jamali - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
mahanje - nearest hotel: No hotel found
berriane - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
eilat - nearest hotel: Melony Club
hawaiian paradise park - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
ra's bayrut - nearest hotel: ثري أو ناين
gebeit - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
harnai - nearest hotel: No hotel found
labuan - 

,City,Country,Lat,Lng,Humidity,Hotel Name
14,enewetak,MH,11.3474,162.3373,74,No hotel found
17,kill devil hills,US,36.0307,-75.6760,66,Mariner Days Inn & Suites
30,veraval,IN,20.9000,70.3667,81,Shyam
51,laguna,US,38.4210,-121.4238,48,Holiday Inn Express & Suites
55,fort bragg,US,35.1390,-79.0060,45,Airborne Inn Lodging


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo=True,
                                    color= "City",
                                    size = "Humidity",
                                    tiles="OSM",
                                    hover_cols=['Hotel Name', 'Country']
                                          )
# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)